# Data 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import json
from pandas import json_normalize

df_train = pd.read_json('/content/drive/MyDrive/NTPUIM/NTCIR/FinNUM 3/Dataset/FinNum-3_ConCall_train.json') #Results contain the required data
df_train

,paragraph,target_num,category,offset_start,offset_end,claim
0,Good day and welcome to the Apple Inc. Third Q...,2018.0,date,65,69,0
1,Please note that some of the information you'l...,10.0,other,504,506,0
2,Please note that some of the information you'l...,8.0,other,536,537,0
3,Thank you Nancy and thanks to everyone for joi...,53.3,money,212,216,0
4,Thank you Nancy and thanks to everyone for joi...,3.0,date,243,244,0
...,...,...,...,...,...,...
8332,Our momentum in pharmacy care services was exc...,2018.0,date,56,60,0
8333,Our momentum in pharmacy care services was exc...,98.0,quantity_relative,102,104,0
8334,Our momentum in pharmacy care services was exc...,2020.0,date,282,286,0
8335,In care delivery our clinical leaders are appl...,99.0,quantity_relative,212,214,0


In [ ]:
df_dev = pd.read_json('/content/drive/MyDrive/NTPUIM/NTCIR/FinNUM 3/Dataset/FinNum-3_ConCall_dev.json') #Results contain the required data
df_dev

,paragraph,target_num,category,offset_start,offset_end,claim
0,In care delivery our clinical leaders are appl...,80.0,other,373,375,0
1,We achieved significantly lower total medical ...,30.0,relative,139,141,0
2,By 2030 there will be over 80 million people i...,2030.0,date,3,7,0
3,By 2030 there will be over 80 million people i...,80.0,quantity_absolute,27,29,1
4,By 2030 there will be over 80 million people i...,3.0,quantity_absolute,62,63,0
...,...,...,...,...,...,...
1186,Now let's move to a summary of the consolidate...,126.0,money,220,223,0
1187,On a comparable basis excluding divestitures a...,2.0,relative,115,116,0
1188,On a comparable basis excluding divestitures a...,45.1,absolute,226,230,0
1189,On a comparable basis excluding divestitures a...,35.7,absolute,283,287,0


In [ ]:
df_test = pd.read_json('/content/drive/MyDrive/NTPUIM/NTCIR/FinNUM 3/Dataset/FinNum-3_ConCall_test.json') #Results contain the required data
df_test

,paragraph,target_num,offset_start,offset_end
0,Let's turn now to our wireless results on slid...,6.0,48,50
1,In the fourth quarter total wireless operating...,1.7,65,68
2,In the fourth quarter total wireless operating...,23.8,74,78
3,In the fourth quarter total wireless operating...,87.5,187,191
4,In the fourth quarter total wireless operating...,1.9,213,216
...,...,...,...,...
2378,We're also combining the project organizations...,2.0,415,416
2379,With the increased upside particularly in the ...,30.0,88,90
2380,With the increased upside particularly in the ...,2019.0,102,106
2381,With the increased upside particularly in the ...,2.0,129,130


In [ ]:
a = df_train.iloc[1,0]
a

"Please note that some of the information you'll hear during our discussion today will consist of forward-looking statements including without limitation those regarding revenue gross margin operating expenses other income and expense taxes capital allocation share repurchases dividends and future business outlook. Actual results or trends could differ materially from our forecast. For more information please refer to the risk factors discussed in Apple's most recently filed periodic reports on Form 10-K and Form 10-Q and the Form 8-K filed with the SEC today along with the associated press release."

#Model

In [ ]:
pip install transformers

     |████████████████████████████████| 3.4 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 895 kB 48.1 MB/s 
     |████████████████████████████████| 61 kB 444 kB/s 
     |████████████████████████████████| 3.3 MB 26.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Tokenizer 


In [ ]:
import torch
from transformers import RobertaModel, RobertaConfig, RobertaTokenizer

pretrained_weights = 'roberta-base'
tokenizer  = RobertaTokenizer.from_pretrained(pretrained_weights)
model =  RobertaModel.from_pretrained(pretrained_weights)

special_token_dict = {'additional_special_token'  : ['<#>','<$>']}
print(tokenizer.encode('<122> <2333>'))

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[0, 41552, 21948, 15698, 28696, 1922, 3103, 15698, 2]


In [ ]:
configuration = RobertaConfig()
model = RobertaModel(configuration)
configuration = model.config
print(configuration)

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [ ]:
print(tokenizer.SPECIAL_TOKENS_ATTRIBUTES)

['bos_token', 'eos_token', 'unk_token', 'sep_token', 'pad_token', 'cls_token', 'mask_token', 'additional_special_tokens']


In [ ]:
special_token_dict = {'additional_special_tokens'  : ['<122>','<2333>'] }
tokenizer.add_special_tokens(special_token_dict)
print(tokenizer.additional_special_tokens)
print(tokenizer.additional_special_tokens_ids)

['<122>', '<2333>']
[50265, 50266]


In [ ]:
print(tokenizer.encode('<122> <2333>'))
# print(tokenizer.encode('<0> this <1> '))
# print(tokenizer.encode('1'))

[0, 50265, 50266, 2]


In [ ]:
tokenizer.vocab_size

50265

In [ ]:
tokenizer.special_tokens_map

{'additional_special_tokens': ['<122>', '<2333>'],
 'bos_token': '<s>',
 'cls_token': '<s>',
 'eos_token': '</s>',
 'mask_token': '<mask>',
 'pad_token': '<pad>',
 'sep_token': '</s>',
 'unk_token': '<unk>'}

In [ ]:
MAX_LEN = 10
sentence1 = '<0> this <1>.'

encoded_dict = tokenizer.encode_plus(
            sentence1,                
            add_special_tokens = True,
            max_length = MAX_LEN,     
            pad_to_max_length = True,
            return_attention_mask = True,  
            return_tensors = 'pt' # return pytorch tensors
       )


encoded_dict

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


{'input_ids': tensor([[    0, 50266,  9226, 50267,     4,     2,     1,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}

In [ ]:
input_ids = encoded_dict['input_ids'][0]
att_mask = encoded_dict['attention_mask'][0]
print(input_ids)
print(att_mask)

tensor([    0, 50266,  9226, 50267,     4,     2,     1,     1,     1,     1])
tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0])


In [ ]:
a = tokenizer.decode(input_ids,
                skip_special_tokens=False)
print(a)

<s> <0> this <1>.</s><pad><pad><pad><pad>


In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50267, 768)

## Pre-Trained

In [ ]:
print(torch.cuda.is_available())

False


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.cuda()
print(device)
print(model)

cuda
RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout):

In [ ]:
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss

In [ ]:
def prepare_features(seq_1, max_seq_length = 512, 
             zero_pad = False, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq_1)

    ## Truncate
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]
    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)
    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    ## Input Mask 
    input_mask = [1] * len(input_ids)
    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [ ]:
prepare_features("Please note that some of the information youll hear during our discussion today will consist of forward-looking statements including without limitation those regarding revenue gross margin operating expenses other income and expense taxes capital allocation share repurchases dividends and future business outlook. Actual results or trends could differ materially from our forecast. For more information please refer to the risk factors discussed in Apple's most recently filed periodic reports on Form 10-K and Form 10-Q and the Form 8-K filed with the SEC today along with the associated press release.")


(tensor([[    0,  6715,  1591,    14,   103,     9,     5,   335,    47,   890,
           1798,   148,    84,  3221,   452,    40, 19438,     9,   556,    12,
           3690,  1997,   217,   396, 22830,   167,  2624,   903,  4200,  2759,
           1633,  4068,    97,  1425,     8,  5623,  2556,   812, 12278,   458,
           2851,  9222,  9354,  8599,     8,   499,   265,  3839,     4, 30144,
            775,    50,  3926,   115, 10356, 11463,    31,    84,  1914,     4,
            286,    55,   335,  2540,  9115,     7,     5,   810,  2433,  3373,
             11,  1257,    18,   144,   682,  1658, 27185,   690,    15,  8575,
            158,    12,   530,     8,  8575,   158,    12,  1864,     8,     5,
           8575,   290,    12,   530,  1658,    19,     5,  3614,   452,   552,
             19,     5,  3059,  1228,   800,     4,     2]]),
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

In [ ]:
from sklearn.model_selection import train_test_split
def split_dataset(df):
    train_set, x = train_test_split(df, 
        stratify=df_train['claim'],
        test_size=0.2, 
        random_state=42)
    val_set, test_set = train_test_split(x, 
        stratify=x['claim'],
        test_size=0.5, 
        random_state=43)

    return train_set,val_set, test_set



train_data,val_data, test_data = split_dataset(df_train)

In [ ]:
test_data.reset_index(drop=True,inplace = True)

In [ ]:
print(test_data.sample(10))

                                              paragraph  ...  claim
6586  So now let's turn to EMEA. In EMEA Q1 revenue ...  ...      0
7190  In addition both Ibrance and Xeljanz revenues ...  ...      0
7171  Moving on to key takeaways. We delivered solid...  ...      0
1833  Revenue of $5.4 billion was up 6% organically....  ...      0
1933  Turning to Slide 3. This is a game of inches a...  ...      0
1175  At Parks and Resorts revenues increased 6% and...  ...      0
3584  Turning to free cash flow. We expect about $12...  ...      0
4438  In Orthopaedics excluding the impact of acquis...  ...      0
4215  We now expect a full year tax rate of 13% 1 po...  ...      1
7777  We plan to deliver another year of 90% or bett...  ...      0

[10 rows x 6 columns]


In [ ]:
class Intents(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        paragraph = self.data.paragraph[index]
        claim = self.data.claim[index]
        X, _  = prepare_features(paragraph)
        y = self.data.claim[index]
        return X, y
    
    def __len__(self):
        return self.len

In [ ]:
df_train.shape

(8337, 6)

In [ ]:
train_data.shape

(6669, 6)

In [ ]:
training_set = Intents(train_data)
print(training_set.__getitem__(0)[0].shape)

torch.Size([1, 28])


In [ ]:
testing_set = Intents(test_data)
print(testing_set.__getitem__(0)[0].shape)

torch.Size([1, 82])


In [ ]:
training_set.__getitem__(0)

(tensor([[    0, 12350,   183,     8,  2814,     7,     5,  1257,   603,     4,
           7470, 10830, 22528,  2041,   199,  7535,  1033,  2815,  3310,     4,
           2477,    18,   486,    16,   145,  2673,     4,     2]]), 0)

In [ ]:
model(training_set.__getitem__(0)[0])[0].size()

torch.Size([1, 28, 768])

In [ ]:
training_set

In [ ]:
training_loader = DataLoader(training_set, batch_size = 1, shuffle = False, drop_last = False)
testing_loader = DataLoader(testing_set, batch_size = 1, shuffle = True, drop_last = False)

In [ ]:
len(training_loader)

1112

In [ ]:
for i, (sent, label) in enumerate(training_loader):
        print('sent: ',sent)
        print('Label:', label)
        print('i:',i)

sent:  tensor([[[    0, 12350,   183,     8,  2814,     7,     5,  1257,   603,     4,
           7470, 10830, 22528,  2041,   199,  7535,  1033,  2815,  3310,     4,
           2477,    18,   486,    16,   145,  2673,     4,     2]]])
Label: tensor([0])
i: 0
sent:  tensor([[[    0,  6715,  1591,    14,   103,     9,     5,   335,    47,   581,
           1798,   148,    84,  3221,   452,    40, 19438,     9,   556,    12,
           3690,  1997,   217,   396, 22830,   167,  2624,   903,  4200,  2759,
           1633,  4068,    97,  1425,     8,  5623,  2556,   812, 12278,   458,
           2851,  9222,  9354,  8599,     8,   499,   265,  3839,     4, 30144,
            775,    50,  3926,   115, 10356, 11463,    31,    84,  1914,     4,
            286,    55,   335,  2540,  9115,     7,     5,   810,  2433,  3373,
             11,  1257,    18,   144,   682,  1658, 27185,   690,    15,  8575,
            158,    12,   530,     8,  8575,   158,    12,  1864,     8,     5,
           85

KeyError: ignored

In [ ]:
training_loader


In [ ]:
loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-05
optimizer = optim.Adam(params =  model.parameters(), lr=learning_rate)

In [ ]:
inp = training_set.__getitem__(0)[0]
output = model(inp)[0]
print(output.shape)

torch.Size([1, 28, 768])


In [ ]:
pretrained_weights = 'roberta-base'
NUM_LABELS = 2
model =  RobertaModel.from_pretrained(pretrained_weights,num_labels=NUM_LABELS)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [ ]:
max_epochs = 3
model = model.train()
loss_function = CrossEntropyLoss()
for epoch in tqdm_notebook(range(max_epochs)):
    print("EPOCH -- {}".format(epoch))
    for i, (sent, label) in enumerate(training_loader):
        print('sent: ',sent)
        print('Label:', label)
        optimizer.zero_grad()
        sent = sent.squeeze(0)
        if torch.cuda.is_available():
          sent = sent
          label = label
        output = model.forward(sent)[0]
        _, predicted = torch.max(output, 1)
        print('output:', output.shape)
        print('label:', label)
        
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        
        if i%100 == 0:
            correct = 0
            total = 0
            for sent, label in testing_loader:
                sent = sent.squeeze(0)
                if torch.cuda.is_available():
                  sent = sent.cuda()
                  label = label.cuda()
                output = model.forward(sent)[0]
                _, predicted = torch.max(output.data, 1)
                total += label.size(0)
                correct += (predicted.cpu() == label.cpu()).sum()
            accuracy = 100.00 * correct.numpy() / total
            print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.item(), accuracy))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/3 [00:00<?, ?it/s]

EPOCH -- 0
sent:  tensor([[[    0, 12350,   183,     8,  2814,     7,     5,  1257,   603,     4,
           7470, 10830, 22528,  2041,   199,  7535,  1033,  2815,  3310,     4,
           2477,    18,   486,    16,   145,  2673,     4,     2]]])
Label: tensor([0])
output: torch.Size([1, 28, 768])
label: tensor([0])


RuntimeError: ignored

In [ ]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader: